# Production Code

In [ ]:
import os
import re
import numpy as np
import pandas as pd

source_directory = "../surveys/plain_txt/"

## Extract

### Global Variables

In [ ]:
shock_list = ["sequía \/ pausas pluviométricas",
              "lluvias intempestivas", "inundaciones",
              "incendios de maleza", "estrés térmico",
              "otros eventos de clima extremo",
              "pérdidas de ganado por enfermedades animales",
              "problema de acceso a servicios veterinarios",
              "otros eventos extremos de salud",
              "muerte de un miembro activo de la familia",
              "enfermedades incapacitantes o accidente de un miembro activo del hogar",
              "aumento del gasto sanitario",
              "otros eventos extremos de salud",
              "aumento de los precios de los piensos",
              "aumento de los precios de los alimentos",
              "disminución de los precios de venta de ganados \/ productos agrícolas",
              "interrupción de transferencias regulares de otros miembros del hogar",
              "pérdidas significativas de ingresos complementario",
              "pérdida de empleo por parte de un miembro de la familia",
              "pérdidas de cultivos \(invasión de langosta\)",
              "otros eventos económicos extremos",
              "robo de ganado",
              "saqueo de cultivos",
              "conflicto \/ violencia \/ i?n?seguridad \/ expropiación  \(a nivel comunitario\)\*",
              "otros eventos extremos \(especificar\)",
              ""]
# sections: #1: 0-5, #2: 6-8, #3: 9-12, #4: 13-20, #5: 21-23, #6: 24
shock_sections = ["choques climáticos", "choque sanitario en el ganado",
                  "choque sanitario en la familia", "choques économico",
                  "choques por la seguridades", "other_extreme_events"]
# excluding nature_of_change column
column_headers = ["past_15_years_adversely_affected_by_change",
                  "most_important_change", "income_affected",
                  "heritage_affected", "food_production_affected",
                  "food_reserves_affected", "food_purchases_affected",
                  "first_most_important_coping_strategy",
                  "second_most_important_coping_strategy",
                  "third_most_important_coping_strategy"]

### Helper Functions

In [ ]:
def find_between(str_1, str_2, contents):
    regex_between = str_1 + r"([\w\W]*)" + str_2
    regex_search = re.findall(regex_between, contents)

    return regex_search

### Parsing Functions

In [ ]:
def parse_interviewee(contents):
    name_regex = r"nombre de la persona[^\n:]*:[\s]*([^\n]*)\n"
    name_search = re.findall(name_regex, contents)
    if len(name_search) == 1:
        name = name_search[0]
        if name == "3":
            name = "N/A"
    else:
        alt_name_regex = r"nombre de la persona[^\n\(]*\(\w* +([^\n]*)\n"
        alt_name_search = re.findall(alt_name_regex, contents)
        if len(alt_name_search) == 1:
            name = alt_name_search[0]
            if name == "3":
                name = "N/A"
        else:
            name = "UNENCOUNTERED FORMAT"

    name = name.strip()

    return name


def parse_section_7(section_search, shock_list,
                    shock_sections, column_headers):
    section_7_data = []

    if len(section_search) == 1:
        section_string = section_search[0] + ""
        # used to parse only the number for each column
        entry_regex = r"([|\d\w -]+)"
        entry_empty_regex = r"(?<!\d)\|[ _]*\|(?!\d)"

        for i in range(len(shock_list)-1):
            item_dict = {}
            item_1 = shock_list[i]
            item_2 = shock_list[i+1]

            row_search = find_between(item_1, item_2, section_string)

            # sections: #1: 0-5, #2: 6-8, #3: 9-12, #4: 13-20, #5: 21-23,
            #           #6: 24
            shock_section = ""
            if i <= 5:
                shock_section = shock_sections[0]
            elif i >= 6 and i <= 8:
                shock_section = shock_sections[1]
            elif i >= 9 and i <= 12:
                shock_section = shock_sections[2]
            elif i >= 13 and i <= 20:
                shock_section = shock_sections[3]
            elif i >= 21 and i <= 23:
                shock_section = shock_sections[4]
            else:
                shock_section = shock_sections[5]
            item_dict["shock_section"] = shock_section
            item_dict["shock"] = item_1

            # check if survey contains this row, because not all surveys
            # have this row as an option
            if (item_2 == "aumento de los precios de los piensos"
               and len(row_search) == 0):
                item_2 = shock_list[i+2]
                row_search = find_between(item_1, item_2, section_string)
            elif (item_1 == "aumento de los precios de los piensos"
                  and len(row_search) == 0):
                for col in column_headers:
                    item_dict[col] = "ROW NOT AVAILABLE"
                section_7_data.append(item_dict)
                continue

            # if expense row found
            if len(row_search) == 1:
                row_string = row_search[0]
                row_list = row_string.split("\n")
                row_list_len = len(row_list)

                if row_list_len >= 24:
                    count = -2

                    for j in range(row_list_len):
                        if count > 18:
                            break
                        if count >= 0 and count % 2 == 0:
                            col = column_headers[count//2]
                            entry = row_list[j]

                            entry_search = re.findall(entry_regex, entry)
                            entry_empty_search = re.findall(entry_empty_regex,
                                                            entry)
                            if len(entry_empty_search) == 1:
                                item_dict[col] = "NO VALUE ENTERED"
                            elif len(entry_search) == 1 and entry != " ":
                                item_dict[col] = entry_search[0]
                            elif entry == "":
                                item_dict[col] = "EMPTY STRING"
                            else:
                                item_dict[col] = "UNENCOUNTERED FORMAT (ENTRY NOT FOUND)"
                        count += 1

                    # parse otros additional info
                    if item_1 == "otros eventos extremos \(especificar\)":
                        otros_regex = r"otros eventos extremos \(especificar\):? ?([^\n]*)"
                        otros_search = re.findall(otros_regex, section_string)

                        if len(otros_search) == 1:
                            otros_string = otros_search[0]
                            item_dict["other_info"] = otros_string
                    elif item_1 == "otros eventos de clima extremo":
                        otros_regex = r"otros eventos de clima extremo:? ?([^\n]*)"
                        otros_search = re.findall(otros_regex, section_string)

                        if len(otros_search) == 1:
                            otros_string = otros_search[0]
                            item_dict["other_info"] = otros_string
                else:
                    for col in column_headers:
                        item_dict[col] = "UNENCOUNTERED FORMAT (ROW NOT LONG ENOUGH)"
            else:
                for col in column_headers:
                    item_dict[col] = "UNENCOUNTERED FORMAT (ROW NOT FOUND)"

            section_7_data.append(item_dict)
    else:
        section_7_data = [{}]

    return section_7_data

### Process the Whole File

In [ ]:
def process_files(source_directory, shock_list,
                  shock_sections, column_headers):
    dataset = []

    for process_file in os.listdir(source_directory):
        file_path = os.path.join(source_directory, process_file)

        # with statements automatically control the closing of files
        with open(file_path, "r") as file:
            contents = file.read()
            contents = contents.lower()

            section_start = "sección vii:"
            # empty because it is the last section
            section_end = ""
            # extract only the relevant section
            section_search = find_between(section_start, section_end, contents)

            interviewee = parse_interviewee(contents)
            section_7_data = parse_section_7(section_search, shock_list,
                                             shock_sections, column_headers)

            data_dict = {}

            # check to make sure files contain the right amount of entries
            if len(section_7_data) <= 1:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of section_data: {len(section_7_data)}\n")

            for row in section_7_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

    # convert list to DataFrame
    raw_df = pd.DataFrame(data=dataset)

    return raw_df

## Transform

### Helper Functions

In [ ]:
def clean_shock_section(dataf):
    replace_dict = {"choques climáticos": "climate_shocks",
                    "choque sanitario en el ganado": "sanitary_shock_in_livestock",
                    "choque sanitario en la familia": "health_shock_in_family",
                    "choques économico": "economic_shock",
                    "choques por la seguridades": "security_shock"}
    dataf["shock_section"] = dataf["shock_section"].replace(replace_dict)

    return dataf


def clean_shock(dataf):
    replace_dict = {"sequía \/ pausas pluviométricas": "droughts",
                    "lluvias intempestivas": "untimely_rain",
                    "inundaciones": "floods",
                    "incendios de maleza": "brush_fires",
                    "estrés térmico": "heat_stress",
                    "otros eventos de clima extremo": "other_extreme_weather_events",
                    "pérdidas de ganado por enfermedades animales": "livestock_loss_from_animal_disease",
                    "problema de acceso a servicios veterinarios": "access_to_veterinary_service_problem",
                    "otros eventos extremos de salud": "other_extreme_health_events",
                    "muerte de un miembro activo de la familia": "death_of_an_active_family_member",
                    "enfermedades incapacitantes o accidente de un miembro activo del hogar": "disabling_illness_or_accident_to_an_active_member_of_the_household",
                    "aumento del gasto sanitario": "increase_in_healthcare_spending",
                    "aumento de los precios de los piensos": "increase_in_feed_prices",
                    "aumento de los precios de los alimentos": "increase_in_food_prices",
                    "disminución de los precios de venta de ganados \/ productos agrícolas": "decrease_in_sale_price_of_livestock_and_agricultural_products",
                    "interrupción de transferencias regulares de otros miembros del hogar": "interruption_of_regular_transfers_of_other_household_members",
                    "pérdidas significativas de ingresos complementario": "significant_loss_of_supplemental_income",
                    "pérdida de empleo por parte de un miembro de la familia": "loss_of_employment_by_a_family_member",
                    "pérdidas de cultivos \(invasión de langosta\)": "crop_losses_locust_invasion",
                    "otros eventos económicos extremos": "other_extreme_economic_events",
                    "robo de ganado": "livestock_theft",
                    "saqueo de cultivos": "crop_plunder",
                    "conflicto \/ violencia \/ i?n?seguridad \/ expropiación  \(a nivel comunitario\)\*": "conflict_violence_insecurity_expropriation_at_a_community_level",
                    "otros eventos extremos \(especificar\)": "other_extreme_events"}
    dataf["shock"] = dataf["shock"].replace(replace_dict)

    return dataf


def remove_column_formatting(dataf, col):
    dataf[col] = dataf[col].str.replace("-", "")
    dataf[col] = dataf[col].str.replace("_", "")
    dataf[col] = dataf[col].str.replace("|", "")
    dataf[col] = dataf[col].str.strip()

    return dataf


def clean_past_15_years(dataf):
    col = "past_15_years_adversely_affected_by_change"

    # remove formatting
    dataf = remove_column_formatting(dataf, col)

    replace_dict = {"1": "yes",
                    "2": "no",
                    "si": "yes",
                    "i": "yes",
                    "11": "yes"}

    dataf[col] = dataf[col].replace(replace_dict)

    return dataf


def clean_most_important_change(dataf):
    col = "most_important_change"

    # remove formatting
    dataf = remove_column_formatting(dataf, col)

    replace_dict = {"2 y 3": "2,3",
                    "1 y 3": "1,3"}

    dataf[col] = dataf[col].replace(replace_dict)

    return dataf


def clean_column_3(dataf):
    cols = ["income_affected", "heritage_affected", "food_production_affected",
            "food_reserves_affected", "food_purchases_affected"]

    # remove formatting
    for col in cols:
        dataf = remove_column_formatting(dataf, col)

    replace_dict = {"1": "increase",
                    "2": "decrease",
                    "3": "no_change"}

    for col in cols:
        dataf[col] = dataf[col].replace(replace_dict)

    return dataf


def clean_column_4(dataf):
    cols = ["first_most_important_coping_strategy",
            "second_most_important_coping_strategy",
            "third_most_important_coping_strategy"]

    # remove formatting
    for col in cols:
        dataf = remove_column_formatting(dataf, col)

    replace_dict = {"1": "greater_mobility",
                    "2": "greater_use_of_family_work",
                    "3": "use_of_salaried_work",
                    "4": "greater_adoption_of_more_resistant_animal_speicies",
                    "5": "establishment_of_private_enclosures_for_forage_crops",
                    "6": "establishment_of_private_enclosures_for_market_gardening",
                    "7": "use_insurance_and_request_expert_support",
                    "8": "eat_alternative_foods",
                    "9": "reduce_number_of_meals_per_day",
                    "01": "greater_mobility",
                    "02": "greater_use_of_family_work",
                    "03": "use_of_salaried_work",
                    "04": "greater_adoption_of_more_resistant_animal_speicies",
                    "05": "establishment_of_private_enclosures_for_forage_crops",
                    "06": "establishment_of_private_enclosures_for_market_gardening",
                    "07": "use_insurance_and_request_expert_support",
                    "08": "eat_alternative_foods",
                    "09": "reduce_number_of_meals_per_day",
                    "10": "reduce_food_eaten_by_adults_to_benefit_children",
                    "11": "sale_of_animals_depopulation",
                    "12": "sale_of_production_goods",
                    "13": "sale_of_non_productive_assets",
                    "14": "sale_of_food_reserves",
                    "15": "decrease_in_the_sale_of_animals",
                    "16": "other_AGRs",
                    "17": "complementary_job_search",
                    "18": "help_from_family_members",
                    "19": "help_from_community_and_associations",
                    "20": "loans",
                    "21": "state_received_aid",
                    "22": "ngo_and_project_support",
                    "23": "unusual_short_migration_<_6_months",
                    "24": "unusual_long_migration_>_6_months",
                    "25": "definitive_migration",
                    "26": "children_trust_in_foster_families",
                    "27": "other_strategy",
                    "28": "no_strategy",}

    for col in cols:
        dataf[col] = dataf[col].replace(replace_dict)

    return dataf

### Pipeline Functions

In [ ]:
def start_pipeline(dataf):
    return dataf.copy()


def handle_null_data(dataf):
    # fill missing values with null
    dataf = dataf.replace("NO VALUE ENTERED", np.NaN)
    dataf = dataf.replace("EMPTY STRING", np.NaN)
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = ((isnull_sum >= 11)
                   | ((isnull_sum >= 9)
                      & ((dataf["past_15_years_adversely_affected_by_change"] == "no")
                         | (dataf["past_15_years_adversely_affected_by_change"].isnull())))
                   | ((isnull_sum >= 9)
                      & ((dataf["past_15_years_adversely_affected_by_change"] == "yes"))))
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf


def clean_values(dataf):
    dataf = clean_shock_section(dataf)
    dataf = clean_shock(dataf)
    dataf = clean_past_15_years(dataf)
    dataf = clean_most_important_change(dataf)
    dataf = clean_column_3(dataf)
    dataf = clean_column_4(dataf)

    return dataf

### Create Raw and Cleaned DataFrame

In [ ]:
raw_df = process_files(source_directory, shock_list,
                       shock_sections, column_headers)

clean_df = (raw_df
            .pipe(start_pipeline)
            .pipe(clean_values)
            .pipe(handle_null_data))

## Other
### Functions to Check the Implementation

In [ ]:
def get_dropped_df(raw_dataf, clean_dataf):
    raw_indexes = raw_dataf.index
    clean_indexes = clean_dataf.index

    dropped_indexes = raw_indexes[~raw_indexes.isin(clean_indexes)]
    dropped_df = raw_dataf.loc[dropped_indexes].copy()

    return dropped_df

### Create Dropped DataFrame
Create the `dropped_df` and collect the rows which contain no non-null values

In [ ]:
dropped_df = get_dropped_df(raw_df, clean_df)

# get rows that were dropped but do not contain non-null values
# (only "EMPTY STRING")
check_filter = (dropped_df == "EMPTY STRING").sum(axis=1) == 10
null_df = dropped_df.loc[check_filter, ["filename", "shock"]]

### Check Dropped Row

In [ ]:
def get_dropped_row(section_search, shock, shock_list):
    dropped_row = {}

    if len(section_search) == 1:
        section_string = section_search[0] + ""

        dropped_row["shock"] = shock

        shock_index = shock_list.index(shock)
        item_1 = shock_list[shock_index]
        item_2 = shock_list[shock_index + 1]

        row_search = find_between(item_1, item_2, section_string)

        # if expense row found
        if len(row_search) == 1:
            row_string = row_search[0]
            row_list = row_string.split("\n")
            row_list_len = len(row_list)

            if row_list_len >= 24:
                count = -2
                for j in range(row_list_len):
                    if count > 18:
                        break
                    if count >= 0:
                        col = str(count)
                        entry = row_list[j]
                        entry = entry.strip()
                        dropped_row[col] = entry
                    count += 1

            else:
                for j in range(19):
                    dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
        else:
            for j in range(19):
                dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
    else:
        dropped_row = {}

    return dropped_row

### Check Dropped Rows for Parsing Errors

In [ ]:
def check_dropped_rows(source_directory, shock_list, s):
    process_file = s["filename"]
    shock = s["shock"]

    check_dict = {}

    file_path = os.path.join(source_directory, process_file)

    # with statements automatically control the closing of files
    with open(file_path, "r") as file:
        contents = file.read()
        contents = contents.lower()

        section_start = "sección vii:"
        # empty because it is the last section
        section_end = ""
        section_search = find_between(section_start, section_end, contents)

        dropped_row = get_dropped_row(section_search, shock, shock_list)

        # check to make sure files contain the right amount of entries
        if len(dropped_row) <= 1:
            print(f"Unencountered Format: {process_file}\n"
                  + f"Length of data: {len(dropped_row)}\n")

        check_dict["filename"] = process_file

        for key, value in dropped_row.items():
            check_dict[key] = value

    # convert list to Series
    check_series = pd.Series(data=check_dict)

    return check_series

In [ ]:
check_df = null_df.apply(lambda s: check_dropped_rows(source_directory,
                                                      shock_list, s),
                         axis=1)

In [ ]:
def drop_null_check(dataf):
    # fill missing values with null
    dataf = dataf.replace("-", np.NaN)
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = isnull_sum >= 19
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf

In [ ]:
non_null_df = (check_df
               .pipe(start_pipeline)
               .pipe(drop_null_check))

In [ ]:
non_null_df

## Load
### Export to CSV

In [ ]:
if non_null_df.shape[0] == 0:
    datasets_directory = "../datasets/"
    filename = "section_7.csv"
    file_path = os.path.join(datasets_directory, filename)

    clean_df.to_csv(file_path, index=False, na_rep="null")

    print(f"Exported to {filename}")
else:
    print("Error: Dropped rows contain non-null values")

***
# Testing Code
### View the Data

In [ ]:
clean_df.head(50)

### Check for Parsing Errors

In [ ]:
cols_to_check = clean_df.columns
cols_to_drop = ["filename", "interviewee", "shock_section", "shock",
                "past_15_years_adversely_affected_by_change",
                "most_important_change"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(clean_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Check Dropped Rows
Another way to check is by opening the variable inspector and manually scrolling
through the `df_dropped` DataFrame. 

The code below tells us whether or not there are files that contain no information for this section. If raw file count is the same as clean file count, then no files were dropped.

In [ ]:
print("Raw File Count: {}".format(len(raw_df["filename"].value_counts())))
print("Clean File Count: {}".format(len(clean_df["filename"].value_counts())))

If there are files that are completely dropped, find those files and look at them to check for parsing errors.

In [ ]:
raw_file_names = raw_df["filename"].value_counts().index
clean_file_names = clean_df["filename"].value_counts().index

raw_file_names[~raw_file_names.isin(clean_file_names)]

Check the rows value counts to see if any rows are being dropped that should not be.

In [ ]:
cols_to_check = dropped_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(dropped_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Duplicate Rows

In [ ]:
df_columns = clean_df.columns[clean_df.columns != "filename"]
clean_df[clean_df.loc[:, df_columns].duplicated(keep=False)]